# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [18]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn

import numpy as np
import time

import math
import random

from torch.utils.data import random_split, Dataset, DataLoader

In [ ]:
def set_random_seed():
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    np.random.seed(42)
    random.seed(42)

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

## Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
$$ P(раму | мама, мыла) * P(папу | мама, мыла) $$
$$ P(столу | дорога, ложка, к) * P(обеду | дорога, ложка, к) $$
$$ P(Евпатий | меня, зовут) * P(Ваня | меня, зовут) $$
$$ P(журналы | я, часто ,читаю) * P(комиксы | я, часто ,читаю) $$
Попробуйте объяснить выбор для каждого из примеров.

Ответ : $$ P(раму | мама, мыла) >  P(папу | мама, мыла) (известный стих) $$
$$ P(столу | дорога, ложка, к) < P(обеду | дорога, ложка, к) (известная пословица) $$
$$ P(Евпатий | меня, зовут) < P(Ваня | меня, зовут) (имя Ваня популярнее) $$
$$ P(журналы | я, часто ,читаю) > P(комиксы | я, часто ,читаю) $$

Если для некоторых из примеров проставить знаки достаточно просто, то на некоторые сложно сказать, какой ответ верный. Мы принимаем решение для данного задания исходя их опыта использования русского языка. Мы много читали на русском и слушали огромное количество русской речи. Обучение языковых моделей происходит по схожему принципу. 

Мы хотим показать модели столько текстов, сколько можем и надеемся, что она наберется достаточно опыта, чтобы расставлять такие знаки неравества максимально схоже с человеком.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распознавании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [ ]:
!wget -O small_corp_for_test.txt nevbros.ru/small_corp_for_test.txt 

--2021-12-20 14:37:01--  http://nevbros.ru/small_corp_for_test.txt
Resolving nevbros.ru (nevbros.ru)... 213.85.168.61
Connecting to nevbros.ru (nevbros.ru)|213.85.168.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56028286 (53M) [text/plain]
Saving to: ‘small_corp_for_test.txt’

small_corp_for_test 100%[===================>]  53.43M  43.9MB/s    in 1.2s    

2021-12-20 14:37:02 (43.9 MB/s) - ‘small_corp_for_test.txt’ saved [56028286/56028286]



In [ ]:
path = 'small_corp_for_test.txt'
file = open(path, 'r', encoding="UTF-8")
data = file.readlines()
file.close()
len(data)

700000

In [ ]:
data

['добро\n',
 'кого\n',
 'капитан\n',
 'нет\n',
 'зачем\n',
 'что происходит\n',
 'что такое\n',
 'рассказ\n',
 'никому\n',
 'ну что\n',
 'кто\n',
 'я укажу\n',
 'исполняй\n',
 'ждет\n',
 'он думал\n',
 'в броне\n',
 'отец\n',
 'быстро\n',
 'ну жил\n',
 'что\n',
 'здоров\n',
 'что с тобой\n',
 'иортэ\n',
 'как\n',
 'ну как\n',
 'ну что ж\n',
 'ах да\n',
 'сударь\n',
 'утонул\n',
 'меня\n',
 'спросил он\n',
 'ну-ка\n',
 'увы\n',
 'но\n',
 'простите\n',
 'вот-вот\n',
 'быстро\n',
 'ждать\n',
 'зачем\n',
 'зачем\n',
 'по пути\n',
 'он здесь\n',
 'ив\n',
 'а другой\n',
 'да нет же\n',
 'случайно\n',
 'ну что ж\n',
 'а как же\n',
 'капри\n',
 'нет\n',
 'я просто\n',
 'уже день\n',
 'сонька\n',
 'ого\n',
 'э-э-э\n',
 'за что\n',
 'как\n',
 'а\n',
 'вот видишь\n',
 'боже\n',
 'а тебе\n',
 'кто\n',
 'не было\n',
 'спросил он\n',
 'не ври\n',
 'ладно\n',
 'крикнул гарри\n',
 'сказал я\n',
 'спросил я\n',
 'посмотри\n',
 'стрела\n',
 'ждешь чего\n',
 'нагнись\n',
 'когда\n',
 'а я\n',
 'марина\n'

In [ ]:
data = [s.rstrip('\n') for s in data]
data = [s.rstrip('\t') for s in data]

In [ ]:
data

['добро',
 'кого',
 'капитан',
 'нет',
 'зачем',
 'что происходит',
 'что такое',
 'рассказ',
 'никому',
 'ну что',
 'кто',
 'я укажу',
 'исполняй',
 'ждет',
 'он думал',
 'в броне',
 'отец',
 'быстро',
 'ну жил',
 'что',
 'здоров',
 'что с тобой',
 'иортэ',
 'как',
 'ну как',
 'ну что ж',
 'ах да',
 'сударь',
 'утонул',
 'меня',
 'спросил он',
 'ну-ка',
 'увы',
 'но',
 'простите',
 'вот-вот',
 'быстро',
 'ждать',
 'зачем',
 'зачем',
 'по пути',
 'он здесь',
 'ив',
 'а другой',
 'да нет же',
 'случайно',
 'ну что ж',
 'а как же',
 'капри',
 'нет',
 'я просто',
 'уже день',
 'сонька',
 'ого',
 'э-э-э',
 'за что',
 'как',
 'а',
 'вот видишь',
 'боже',
 'а тебе',
 'кто',
 'не было',
 'спросил он',
 'не ври',
 'ладно',
 'крикнул гарри',
 'сказал я',
 'спросил я',
 'посмотри',
 'стрела',
 'ждешь чего',
 'нагнись',
 'когда',
 'а я',
 'марина',
 'спросил я',
 'ого',
 'а-а',
 'вот как',
 'у кого',
 'никогда',
 'а',
 'уф',
 'это реми',
 'я',
 'все',
 'да',
 'нет',
 'хорошо',
 'никто',
 'меня',


## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длины для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длину текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [ ]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчм яжлйвцыэь-шхющёъф][ '
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        text = text.lower()
        text += (window_size - len(text)) * '_'
        tokenizer = [self.token2ind[t] for t in text]
        return tokenizer[:-1], tokenizer[1:]

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

In [ ]:
token = "[]"
data = [token[:1] + t + token[1:] for t in data]
data[:10]

['[добро]',
 '[кого]',
 '[капитан]',
 '[нет]',
 '[зачем]',
 '[что происходит]',
 '[что такое]',
 '[рассказ]',
 '[никому]',
 '[ну что]']

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длину текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [ ]:
THRESHOLD = 128

data = [text for text in data if len(text) <= THRESHOLD]
print(f'data_size = {len(data)}')

train_size = int(0.85 * (len(data)))
test_size = len(data) - train_size
train_dataset, test_dataset = random_split(data, [train_size, test_size])
print(f'train_size = {train_size},',
      f'test_size = {test_size}')

data_size = 683438
train_size = 580922, test_size = 102516


## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [ ]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = 128):
      self.x = [preproc.preprocess(text, win_size) for text in x]

    def __len__(self):
      return len(self.x)
    
    def __getitem__(self, idx):
      return torch.Tensor(self.x[idx][0]).int(), torch.Tensor(self.x[idx][1]).int()

In [ ]:
preproc = Preprocessor()
train_dataset = TextDataset(train_dataset, preproc)
test_dataset = TextDataset(test_dataset, preproc)

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде используйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_size = 64, encoders = 6, n_heads = 1):
        super(LanguageModel, self).__init__()
        self.emb = nn.Embedding(vocab_size, embedding_size)
        self.pe = PositionalEncoding(embedding_size)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(embedding_size, n_heads);
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, encoders);
        self.decoder = nn.Sequential(
            nn.Linear(embedding_size, int(0.8 * embedding_size)),
            nn.ReLU(),
            nn.Linear(int(0.8 * embedding_size), vocab_size),                                    
        )
    
    def forward(self, x, src_mask):
        x = self.pe(self.emb(x)) # emb, then pe
        x = x.transpose(1, 0)
        x = self.transformer_encoder(x , src_mask) # transformer encoder with mask
        x = self.decoder(x) # decoder
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [ ]:
vocab_size = len('_добсркгаупитнезчм яжлйвцыэь-шхющёъф][ ')
model = LanguageModel(vocab_size=vocab_size)

Маскирование - пытаемся предсказать рандомное слово в текстовой последовательности (предсказания для i-ой позиции зависят только от известных уже выходов позиций меньше i)


## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [19]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset, win_size, optimizer):
        
        self.model = model
        self.mask = self.model.generate_square_subsequent_mask(win_size)
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size=self.train_batch_size,
            num_workers=1,
            pin_memory=True,
            shuffle=True,
        )
        self.test_dataloader = DataLoader(
            dataset=test_dataset,
            batch_size=self.test_batch_size,
            num_workers=1,
            pin_memory=True,
            shuffle=True,
        )
        self.train_dataloader_size = len(self.train_dataloader)
        self.test_dataloader_size = len(self.test_dataloader)
        
        self.device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        self.mask.to(self.device)
        self.criterion = torch.nn.CrossEntropyLoss(ignore_index=0) # используйте CrossEntrophyLoss, передайте в качетсве параметра 
                             # ignore index индекс символа _, чтобы модель не штрафовалась за то
                             # что идет после закрывающего токена
        
        self.optimizer = optimizer
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader:
            x, y = batch
            
            x = x.to(self.device)
            y = y.to(self.device)

            self.optimizer.zero_grad()
            predicted = self.model(x, self.mask.to(self.device))
            loss = self.criterion(predicted.permute(1, 2, 0), y.permute(1, 0).long())
            counted_loss += loss.item()
            loss.backward()
            self.optimizer.step()

            it += x.shape[0] * x.shape[1]
            step += 1
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch

            x = x.to(self.device)
            y = y.to(self.device)

            predicted = self.model(x, self.mask.to(self.device))
            loss = self.criterion(predicted.permute(1, 2, 0), y.permute(1, 0).long())
            counted_loss += loss.item()

            it += x.shape[0] * x.shape[1]
            step += 1
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [ ]:
vocab_size = len('_добсркгаупитнезчм яжлйвцыэь-шхющёъф][ ')
model = LanguageModel(vocab_size=vocab_size, embedding_size=32)
trainer = Trainer(model, train_dataset, test_dataset, 127, torch.optim.Adam(model.parameters(), lr=5e-4))

In [ ]:
trainer.train(2)

Train epoch 1 | Step 1000/9077 | Counted loss 2632.086046218872 | ppl 1.0003238819224605 | time 127.61999320983887 | 
Train epoch 1 | Step 2000/9077 | Counted loss 2283.0728640556335 | ppl 1.000280929323339 | time 126.90019345283508 | 
Train epoch 1 | Step 3000/9077 | Counted loss 2082.2387293577194 | ppl 1.000256213763174 | time 126.95748114585876 | 
Train epoch 1 | Step 4000/9077 | Counted loss 1966.4064384698868 | ppl 1.0002419591933975 | time 126.55339574813843 | 
Train epoch 1 | Step 5000/9077 | Counted loss 1902.3840562105179 | ppl 1.000234080549099 | time 126.91560697555542 | 
Train epoch 1 | Step 6000/9077 | Counted loss 1848.041338443756 | ppl 1.0002273931405565 | time 126.72219657897949 | 
Train epoch 1 | Step 7000/9077 | Counted loss 1810.7078355550766 | ppl 1.0002227989099326 | time 126.55136799812317 | 
Train epoch 1 | Step 8000/9077 | Counted loss 1784.039603114128 | ppl 1.0002195171517714 | time 126.6609890460968 | 
Train epoch 1 | Step 9000/9077 | Counted loss 1756.9427

In [21]:
model = LanguageModel(vocab_size=vocab_size, embedding_size=32)
trainer = Trainer(model, train_dataset, test_dataset, 127, torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9))
trainer.train(2)

Train epoch 1 | Step 1000/9077 | Counted loss 3061.353116750717 | ppl 1.0003767137927566 | time 121.63815402984619 | 
Train epoch 1 | Step 2000/9077 | Counted loss 2779.5066888332367 | ppl 1.0003420253435786 | time 121.35005354881287 | 
Train epoch 1 | Step 3000/9077 | Counted loss 2699.198506832123 | ppl 1.0003321415772695 | time 121.04561233520508 | 
Train epoch 1 | Step 4000/9077 | Counted loss 2658.782860517502 | ppl 1.0003271675406655 | time 121.33617877960205 | 
Train epoch 1 | Step 5000/9077 | Counted loss 2630.7254388332367 | ppl 1.0003237144706951 | time 121.22323656082153 | 
Train epoch 1 | Step 6000/9077 | Counted loss 2608.933310508728 | ppl 1.0003210324882237 | time 120.90603303909302 | 
Train epoch 1 | Step 7000/9077 | Counted loss 2591.910663843155 | ppl 1.000318937496386 | time 121.42793774604797 | 
Train epoch 1 | Step 8000/9077 | Counted loss 2579.2012615203857 | ppl 1.0003173733421182 | time 121.38871932029724 | 
Train epoch 1 | Step 9000/9077 | Counted loss 2566.077

In [33]:
model = LanguageModel(vocab_size=vocab_size, embedding_size=128)
trainer = Trainer(model, train_dataset, test_dataset, 127, torch.optim.Adam(model.parameters(), lr=5e-4))
trainer.train(2)

Train epoch 1 | Step 1000/9077 | Counted loss 2231.3110613822937 | ppl 1.0002745592225495 | time 197.89398217201233 | 
Train epoch 1 | Step 2000/9077 | Counted loss 1726.1254737377167 | ppl 1.0002123903510638 | time 197.7868800163269 | 
Train epoch 1 | Step 3000/9077 | Counted loss 1602.101330757141 | ppl 1.0001971283508078 | time 197.23089456558228 | 
Train epoch 1 | Step 4000/9077 | Counted loss 1534.5032403469086 | ppl 1.000188810051748 | time 196.8667664527893 | 
Train epoch 1 | Step 5000/9077 | Counted loss 1497.874790072441 | ppl 1.0001843027580044 | time 196.92610263824463 | 
Train epoch 1 | Step 6000/9077 | Counted loss 1465.7907358407974 | ppl 1.0001803546890975 | time 196.81869173049927 | 
Train epoch 1 | Step 7000/9077 | Counted loss 1447.922952413559 | ppl 1.000178155994914 | time 196.79463052749634 | 
Train epoch 1 | Step 8000/9077 | Counted loss 1430.355887055397 | ppl 1.0001759943098687 | time 196.81097960472107 | 
Train epoch 1 | Step 9000/9077 | Counted loss 1409.16345

Вывод сохранила не от всех проведенных экспериментов, но итог можно сделать такой: лучший оптимайзер - Адам с параметром lr=5e-4 (по экспериментам выше видно, насколько лучше SGD), в модели лучше всего брать embedding_size=128

## Задание 9 (1 балл)
Итак, давайте попробуем погенерировать текст нашей сеткой. Закончите функцию по генерации текста. Попробуйте сгенерировать какой-нибудь текст. Помните, что если вы хотите генерировать текст с нуля, то вы должны передать в качестве текста только токен start.
Прекратите генерировать текст, если модель выдала токен end или длина текста больше 150.

In [34]:
def generate_text(text):
    x = []
    
    for letter in text:
        x.append(preproc.token2ind[letter])
    #x = torch.from_numpy(np.array(x))
    x = torch.Tensor([x]).int()
    win_size = len(x[0])
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    x = x.to(device)
    mask = model.generate_square_subsequent_mask(win_size).to(device)
    pred = model(x, mask)[0]
    ind = torch.argmax(pred[-1])
    
    text += preproc.ind2token[ind.item()]
    
    if preproc.ind2token[ind.item()] == ']' or len(text) >= 150:
        return text
    else:
        return generate_text(text)

In [35]:
generate_text('[детст')

'[детствует по поводу поводу по какому вопросу]'

In [36]:
generate_text('[школ')

'[школа подавали на сайте в принципе в принципе в принципе в принципе по поводу стоимости в продвижении в поисковом случае в сайте]'

In [37]:
generate_text('друз')

'друз ана я слушаю вас]'

## Задание 10* (Задание - бонус, 5 баллов за реализацию при условии, что сделаны прошлые задания)
Давайте вспомним, что такое transfer learning. Мы хотим использовать уже предобученные эмбединги для нашей сети, чтобы наша сеть обучалась быстрее. Давайте попробуем обучить новую модель на уровне слов, а не символов, но для упрощения задачи используем предобученный слой из библиотеки Natasha, а вернее, ее блок Navec.

[Изучите](https://github.com/natasha/navec) то, как вставить слой в вашу нейронную сеть.

Теперь мы хотим, чтобы на вход модели подавались слова, модифицируйте ваш датасет. Возвращайте теперь номер слова в словаре navec.

In [ ]:
class TextDataset_Navec(torch.utils.data.Dataset):
    
    def __init__(self, x, win_size = 128):
        # YOUR CODE HERE
        self.navec = ...
        ################
    
    def __len__(self):
        # YOUR CODE HERE
        ################
    
    def __getitem__(self, idx):
        # YOUR CODE HERE
        ################

Немного модифицируем модель. Теперь нам не нужны слои с трансформером, так как весь механизм внимания уже заложен в ембедингах. Давайте попробуем просто пройтись линейной головой над эмбедингами. Выберите параметры самостоятельно.

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self):
        super(LanguageModel, self).__init__()
        self.emb_navec = ...
        self.head = ...
    
    def forward(self, x):
        x = ... # emb
        x = ... # head
        return x

Теперь дело за малым! Надо немного модифицировать класс обучения, так как мы не используем маскирование, после чего можно приступить к тесту!

In [ ]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = ...
        self.test_dataloader = ...
        self.train_dataloader_size = ...
        self.test_dataloader_size = ...
        
        self.device = 'cuda:0'
        self.criterion = ... 
        
        self.optimizer = ...
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader:
            x, y = batch
            # YOUR CODE HERE
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Запустите обучение. 

In [ ]:
# YOUR CODE HERE
###############